In [1]:
!pip install -q openai-whisper --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import whisper
import torch
import torchaudio
from google.colab import files

# Step 1: Upload audio file

In [8]:
print("📤 Upload your audio file:")
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print(f"✅ File uploaded: {audio_path}")

📤 Upload your audio file:


Saving mixed.mp3 to mixed.mp3
✅ File uploaded: mixed.mp3


# Step 2: Preprocessing

In [9]:
def load_and_normalize(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    # normalize waveform between -1 and 1
    waveform = waveform / waveform.abs().max()
    return waveform, sample_rate

waveform, sr = load_and_normalize(audio_path)
print(f"🎧 Loaded '{audio_path}' with sample rate: {sr}")

🎧 Loaded 'mixed.mp3' with sample rate: 48000


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

# Step 3: Load model + custom options


In [5]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:40<00:00, 75.7MiB/s]


In [6]:
# Optional inputs
language = input("🌍 Enter language ('en', 'ar', leave empty for mixed): ") or None
prompt = input("💬 Enter a prompt for context (or leave empty): ") or None

🌍 Enter language ('en', 'ar', leave empty for mixed): 
💬 Enter a prompt for context (or leave empty): The speaker mixes Egyptian Arabic and English. Transcribe exactly what is spoken. Do not translate. Keep English words in English letters and Egyptian Arabic in Arabic letters, as pronounced in the dialect.


# Step 4: Transcribe

In [11]:
print("🧠 Transcribing...")
result = model.transcribe(
    audio_path,
    language=language,
    prompt=prompt,
    temperature=0.0,     # deterministic decoding
    beam_size=5,         # explore multiple options
    best_of=5,           # helps find best output
    patience=2,          # allow slightly longer beam search
    condition_on_previous_text=True,  # better context handling
)

print("\n📝 Transcription:")
print(result["text"])

🧠 Transcribing...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:
 هاي يا ادم عملت رن الويسبر على الاوديو بتاع الميتنج اللي فات؟ اه عملت والرزلت كانت كويسة في الانجليش بس العربي لسه فيه مشاكل شوية طيب جربت جوجل speech to text؟ ايوة بس هو ما يفهمش اي كلمة عربي تقريباً بيحاول يترجمها او بيهملها خالص امم اكسبيتد طب جربنا اوتر؟ لسه بس عامل تاست عليه النهاردة عايزين يشوف هل هيقدر يفصل ما بين الspeakers ولا لأ true speaker diarization مهمة جداً في البروجيكت ده especially لو عايزين نعمل حاجة زي meeting assistant بالزبط وانا بفكر كمان نزود step نعمل زي noise suppression ablet transcription فكرة حلوة جداً ممكن نجربها كpre-processing step ونشوف هل الword error rate هتتحسن؟ تمام خلينا نكتب الرزلت دي في ال report بتاع sprint 1 ونوضح الlimitations اللي شفناها agreed نخلصها بكرة نشوفها تمام زيرفور
